In [1]:
import sympy as smp

from sympy.physics.mechanics import *
from sympy.physics.quantum.trace import Tr
from sympy.physics.quantum import *
from sympy.physics.quantum.density import *
from sympy.physics.quantum import IdentityOperator

from sympy import Symbol, Integer
from sympy.physics.quantum import Dagger, qapply, represent, InnerProduct, Commutator
from sympy.physics.quantum.sho1d import RaisingOp, LoweringOp, NumberOp, Hamiltonian, SHOKet, SHOBra

from sympy.physics.quantum import Commutator, Dagger, Operator

import numpy as np
import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science', 'notebook', 'nature'])

from sympy.physics.paulialgebra import Pauli, evaluate_pauli_product
import scipy as scp

In [14]:
import scipy as scp
import scipy.sparse
import scipy.sparse.linalg
from scipy.linalg import expm
from scipy.sparse import csr_array

from scipy.integrate import solve_ivp
from scipy.optimize import fmin
import matplotlib.pyplot as plt
import numpy as np

import functools 
from tqdm import tqdm
import os

from pathlib import Path

import seaborn as sns
import pandas as pd

# Define quantities


In [2]:
J_smp = smp.symbols('J', real=True, positive=True)

In [3]:
X = 1/2 * smp.Matrix([[0, 1], [1, 0]])
Y = 1/2 * smp.Matrix([[0, -smp.I], [smp.I, 0]])
Z = 1/2 * smp.Matrix([[1, 0], [0, -1]])
I = smp.Matrix([[1, 0], [0, 1]])

In [4]:
def spin_j(j, N):
    """
    Returns the spin operators for a spin-j system with N particles.

    Parameters
    ----------
    j : int
        The spin to act on.
    N : int
        The number of particles.

    Returns
    -------
    Sx_j : sympy.Matrix
        The X spin operator acting on spin j.
    Sy_j : sympy.Matrix
        The Y spin operator acting on spin j.
    Sz_j : sympy.Matrix
        The Z spin operator acting on spin j.
    
    """

    Sx = smp.KroneckerProduct()
    Sy = smp.KroneckerProduct()
    Sz = smp.KroneckerProduct()

    true_j = j % N

    if true_j == 0:
        true_j = N

    for i in range(1, N+1):
        if i == true_j:
            Sx = smp.KroneckerProduct(Sx, X)
            Sy = smp.KroneckerProduct(Sy, Y)
            Sz = smp.KroneckerProduct(Sz, Z)
        else:
            Sx = smp.KroneckerProduct(Sx, I)
            Sy = smp.KroneckerProduct(Sy, I)
            Sz = smp.KroneckerProduct(Sz, I)

    return Sx.doit(), Sy.doit(), Sz.doit()

Sx, Sy, Sz = spin_j(4, 4)
Sy

Matrix([
[    0, -0.5*I,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0],
[0.5*I,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0],
[    0,      0,     0, -0.5*I,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0],
[    0,      0, 0.5*I,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0],
[    0,      0,     0,      0,     0, -0.5*I,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0],
[    0,      0,     0,      0, 0.5*I,      0,     0,      0,     0,      0,     0,      0,     0,      0,     0,      0],
[    0,      0,     0,      0,     0,      0,     0, -0.5*I,     0,      0,     0,      0,     0,      0,     0,      0],
[    0,      0,     0,      0,     0,      0, 0.5*I,      0,     0,      0,     0,      0,     0,      0,     0,      0],
[    0,      0,

In [5]:
def Jh_fun(N) :
    """
    Computes the Heat current J_H in the Heisenberg model

    Parameters
    ----------
    N : int
        The number of particles.

    Returns
    -------
    Jh : sympy.Matrix
        The heat current operator.
    """
    Jh = smp.zeros(2**N)
    isfirst = True
    for j in range(1, N+1):
        if isfirst:
            Sxj, Syj, Szj = spin_j(j, N)
            Sxjp1, Syjp1, Szjp1 = spin_j(j+1, N)
            Sxjp2, Syjp2, Szjp2 = spin_j(j+2, N)
            isfirst = False
        else :
            Sxj, Syj, Szj = Sxjp1, Syjp1, Szjp1
            Sxjp1, Syjp1, Szjp1 = Sxjp2, Syjp2, Szjp2
            Sxjp2, Syjp2, Szjp2 = spin_j(j+2, N)

        cross_product = [Syj * Szjp1 - Szj * Syjp1, Szj * Sxjp1 - Sxj * Szjp1, Sxj * Syjp1 - Syj * Sxjp1]
        Jh += cross_product[0] * Sxjp2 + cross_product[1] * Syjp2 + cross_product[2] * Szjp2

    return J_smp**2 * Jh

Jh = Jh_fun(5)
# Jh


In [9]:
def H0_fun(N) :
    """
    Computes the Heisenberg Hamiltonian H_0 of the Heisenberg model

    Parameters
    ----------
    N : int
        The number of particles.

    Returns
    -------
    H0 : sympy.Matrix
        The Heisenberg Hamiltonian operator.
    """
    H0 = smp.zeros(2**N)
    isfirst = True
    for j in range(1, N+1):
        if isfirst:
            Sxj, Syj, Szj = spin_j(j, N)
            Sxjp1, Syjp1, Szjp1 = spin_j(j+1, N)
            isfirst = False
        else :
            Sxj, Syj, Szj = Sxjp1, Syjp1, Szjp1
            Sxjp1, Syjp1, Szjp1 = spin_j(j+1, N)

        H0 += Sxj * Sxjp1 + Syj * Syjp1 + Szj * Szjp1
    return J_smp * H0

H0 = H0_fun(5)
# H0

In [10]:
def Ob_fun(N) :
    """
    Computes the boost operator Ob of the Heisenberg model

    Parameters
    ----------
    N : int
        The number of particles.

    Returns
    -------
    Ob : sympy.Matrix
        The boost operator.
    """
    Ob = smp.zeros(2**N)
    isfirst = True
    for j in range(1, N+1):
        if isfirst:
            Sxj, Syj, Szj = spin_j(j, N)
            Sxjp1, Syjp1, Szjp1 = spin_j(j+1, N)
            isfirst = False
        else :
            Sxj, Syj, Szj = Sxjp1, Syjp1, Szjp1
            Sxjp1, Syjp1, Szjp1 = spin_j(j+1, N)
        Ob += j * (Sxj * Sxjp1 + Syj * Syjp1 + Szj * Szjp1)
    return - smp.I * Ob

Ob = Ob_fun(5)
# Ob

In [11]:
N = 8

C2 = H0_fun(N)
C3 = Jh_fun(N)
Ob = Ob_fun(N)

# # print True if [Jh, H0] == 0 
# display(Jh * H0 - H0 * Jh == smp.zeros(2**N))       

# [Ob, Jh]
C4 = Ob * C3 - C3 * Ob; C4
A = C4 * C2 - C2 * C4; A            

C5 = Ob * C4 - C4 * Ob; C5
B = C5 * C2 - C2 * C5; B            

print('done')       # 3min for N=8


done


In [12]:

B_fun = smp.lambdify((J_smp), A, 'numpy')
B_mat = scp.sparse.csr_array(B_fun(1))

In [14]:
B_mat

<256x256 sparse array of type '<class 'numpy.float64'>'
	with 1984 stored elements in Compressed Sparse Row format>

In [16]:
np.diag(B_mat.toarray(), k=1)

array([ 0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
        0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,
        0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,
        0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
        0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
        0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. , -0.5,
        0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,
       -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,
        0. , -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. , -0.5,  0. ,
        0. ,  0. , -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. , -0.5,
        0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,
       -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,
        0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ,
        0. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0

In [204]:
k = 4
Lk1 = spin_j(k, N)[2]

In [206]:
# Lk1 * C2 * Lk1

In [12]:
Z * X * Z

Matrix([
[     0, -0.125],
[-0.125,      0]])

In [13]:
Z * Y * Z

Matrix([
[       0, 0.125*I],
[-0.125*I,       0]])

In [14]:
Z * Z * Z

Matrix([
[0.125,      0],
[    0, -0.125]])

In [41]:
Lk = Operator('L_k'); display(Lk)
Lk_dagger = Operator('L_k^\dagger'); display(Lk_dagger)
Sz = Operator('S_z'); display(Sz)
Cj = Operator('C_j'); display(Cj)

L_k

L_k^\dagger

S_z

C_j

In [42]:
comm1 = Commutator(Lk_dagger, Cj); comm1
comm2 = Commutator(Cj, Lk); comm2
expr = comm1 * Lk + Lk_dagger * comm2; expr

-[C_j,L_k^\dagger]*L_k + L_k^\dagger*[C_j,L_k]

In [43]:
expr = expr.expand(commutator=True).doit().expand(); expr

-C_j*L_k^\dagger*L_k + 2*L_k^\dagger*C_j*L_k - L_k^\dagger*L_k*C_j

In [46]:
expr.subs({Lk: Sz, Lk_dagger: Sz}).subs({Sz**2 : 1/4})

-0.5*C_j + 2*S_z*C_j*S_z

In [71]:
sigma_x = Pauli(1)
sigma_y = Pauli(2)
sigma_z = Pauli(3)
sigma_p = (sigma_x + smp.I * sigma_y) / 2
sigma_m = (sigma_x - smp.I * sigma_y) / 2

In [50]:
sigma_m

(sigma1 - I*sigma2)/2

In [54]:
evaluate_pauli_product((sigma_p * sigma_m).expand())

1/2 + sigma3/2

In [55]:
evaluate_pauli_product((sigma_m * sigma_p).expand())

1/2 - sigma3/2

In [68]:
(smp.KroneckerProduct(I, I, X, I, I) - smp.I * smp.KroneckerProduct(I, I, Y, I, I)) / 2 == smp.KroneckerProduct(I, I, (X - smp.I * Y)/2, I, I).doit()

True

In [72]:
Sx = Pauli(1) / 2
Sy = Pauli(2) / 2
Sm = (Sx - smp.I * Sy) / 2
Sp = (Sx + smp.I * Sy) / 2

In [74]:
(Sm - Sp).expand()

-I*sigma2/2

In [76]:
(Sm + Sp).expand()

sigma1/2

# N = 2


In [14]:
X1 = smp.KroneckerProduct(X, I); display(X1)
X2 = smp.KroneckerProduct(I, X); display(X2)

Y1 = smp.KroneckerProduct(Y, I); display(Y1)
Y2 = smp.KroneckerProduct(I, Y); display(Y2)

Z1 = smp.KroneckerProduct(Z, I); display(Z1)
Z2 = smp.KroneckerProduct(I, Z); display(Z2)


KroneckerProduct(Matrix([
[  0, 0.5],
[0.5,   0]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[  0, 0.5],
[0.5,   0]]))

KroneckerProduct(Matrix([
[    0, -0.5*I],
[0.5*I,      0]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[    0, -0.5*I],
[0.5*I,      0]]))

KroneckerProduct(Matrix([
[0.5,    0],
[  0, -0.5]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[0.5,    0],
[  0, -0.5]]))

In [15]:
H = J_smp * (X1 * X2 + Y1 * Y2 + Z1 * Z2 + X2 * X1 + Y2 * Y1 + Z2 * Z1) 
H

Matrix([
[0.5*J,      0,      0,     0],
[    0, -0.5*J,  1.0*J,     0],
[    0,  1.0*J, -0.5*J,     0],
[    0,      0,      0, 0.5*J]])

In [16]:
JH = (Y1 * Z2 - Z1 * Y2) * X1 \
        + (- X1 * Z2 + Z1 * X2) * Y1 \
        + (X1 * Y2 - Y1 * X2) * Z1 \
        + (Y2 * Z1 - Z2 * Y1) * X2 \
        + (- X2 * Z1 + Z2 * X1) * Y2 \
        + (X2 * Y1 - Y2 * X1) * Z2

JH

Matrix([
[-0.5*I,      0,      0,      0],
[     0,  0.5*I, -1.0*I,      0],
[     0, -1.0*I,  0.5*I,      0],
[     0,      0,      0, -0.5*I]])

In [17]:
OB = - smp.I * (1 * (X1 * X2 + Y1 * Y2 + Z1 * Z2) \
                + 2 * (X2 * X1 + Y2 * Y1 + Z2 * Z1) )
OB

Matrix([
[-0.75*I,      0,      0,       0],
[      0, 0.75*I, -1.5*I,       0],
[      0, -1.5*I, 0.75*I,       0],
[      0,      0,      0, -0.75*I]])

# N = 3


In [18]:
X1 = smp.KroneckerProduct(X, I, I); display(X1)
X2 = smp.KroneckerProduct(I, X, I); display(X2)
X3 = smp.KroneckerProduct(I, I, X); display(X3)

Y1 = smp.KroneckerProduct(Y, I, I); display(Y1)
Y2 = smp.KroneckerProduct(I, Y, I); display(Y2)
Y3 = smp.KroneckerProduct(I, I, Y); display(Y3)

Z1 = smp.KroneckerProduct(Z, I, I); display(Z1)
Z2 = smp.KroneckerProduct(I, Z, I); display(Z2)
Z3 = smp.KroneckerProduct(I, I, Z); display(Z3)


KroneckerProduct(Matrix([
[  0, 0.5],
[0.5,   0]]), Matrix([
[1, 0],
[0, 1]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[  0, 0.5],
[0.5,   0]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[1, 0],
[0, 1]]), Matrix([
[  0, 0.5],
[0.5,   0]]))

KroneckerProduct(Matrix([
[    0, -0.5*I],
[0.5*I,      0]]), Matrix([
[1, 0],
[0, 1]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[    0, -0.5*I],
[0.5*I,      0]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[1, 0],
[0, 1]]), Matrix([
[    0, -0.5*I],
[0.5*I,      0]]))

KroneckerProduct(Matrix([
[0.5,    0],
[  0, -0.5]]), Matrix([
[1, 0],
[0, 1]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[0.5,    0],
[  0, -0.5]]), Matrix([
[1, 0],
[0, 1]]))

KroneckerProduct(Matrix([
[1, 0],
[0, 1]]), Matrix([
[1, 0],
[0, 1]]), Matrix([
[0.5,    0],
[  0, -0.5]]))

In [19]:
H = J_smp * (X1 * X2 + Y1 * Y2 + Z1 * Z2 \
            + X2 * X3 + Y2 * Y3 + Z2 * Z3 \
            + X3 * X1 + Y3 * Y1 + Z3 * Z1 ) 
H

Matrix([
[0.75*J,       0,       0,       0,       0,       0,       0,      0],
[     0, -0.25*J,   0.5*J,       0,   0.5*J,       0,       0,      0],
[     0,   0.5*J, -0.25*J,       0,   0.5*J,       0,       0,      0],
[     0,       0,       0, -0.25*J,       0,   0.5*J,   0.5*J,      0],
[     0,   0.5*J,   0.5*J,       0, -0.25*J,       0,       0,      0],
[     0,       0,       0,   0.5*J,       0, -0.25*J,   0.5*J,      0],
[     0,       0,       0,   0.5*J,       0,   0.5*J, -0.25*J,      0],
[     0,       0,       0,       0,       0,       0,       0, 0.75*J]])

In [20]:
JH = (Y1 * Z2 - Z1 * Y2) * X3 \
        + (- X1 * Z2 + Z1 * X2) * Y3 \
        + (X1 * Y2 - Y1 * X2) * Z3 \
        + (Y2 * Z3 - Z2 * Y3) * X1 \
        + (- X2 * Z3 + Z2 * X3) * Y1 \
        + (X2 * Y3 - Y2 * X3) * Z1 \
        + (Y3 * Z1 - Z3 * Y1) * X2 \
        + (- X3 * Z1 + Z3 * X1) * Y2 \
        + (X3 * Y1 - Y3 * X1) * Z2

JH

Matrix([
[0,       0,       0,       0,       0,       0,       0, 0],
[0,       0,  0.75*I,       0, -0.75*I,       0,       0, 0],
[0, -0.75*I,       0,       0,  0.75*I,       0,       0, 0],
[0,       0,       0,       0,       0, -0.75*I,  0.75*I, 0],
[0,  0.75*I, -0.75*I,       0,       0,       0,       0, 0],
[0,       0,       0,  0.75*I,       0,       0, -0.75*I, 0],
[0,       0,       0, -0.75*I,       0,  0.75*I,       0, 0],
[0,       0,       0,       0,       0,       0,       0, 0]])

In [22]:
OB = - smp.I * (1 * (X1 * X2 + Y1 * Y2 + Z1 * Z2) \
            + 2 * (X2 * X3 + Y2 * Y3 + Z2 * Z3) \
            + 3 * (X3 * X1 + Y3 * Y1 + Z3 * Z1) ) 
OB

Matrix([
[-1.5*I,      0,      0,      0,      0,      0,      0,      0],
[     0,  1.0*I, -1.0*I,      0, -1.5*I,      0,      0,      0],
[     0, -1.0*I,      0,      0, -0.5*I,      0,      0,      0],
[     0,      0,      0,  0.5*I,      0, -0.5*I, -1.5*I,      0],
[     0, -1.5*I, -0.5*I,      0,  0.5*I,      0,      0,      0],
[     0,      0,      0, -0.5*I,      0,      0, -1.0*I,      0],
[     0,      0,      0, -1.5*I,      0, -1.0*I,  1.0*I,      0],
[     0,      0,      0,      0,      0,      0,      0, -1.5*I]])

In [26]:
C4 = OB * JH - JH * OB
C4 @ H - H @ C4

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0]])

In [7]:
from sympy import SparseMatrix

def csr_to_sparse_matrix(data, indices, indptr, shape):
    """
    Convert CSR arrays to SymPy SparseMatrix.

    Parameters:
    - data: 1-D array containing the non-zero elements of the matrix.
    - indices: 1-D array containing the column indices of the non-zero elements.
    - indptr: 1-D array indicating the start of rows in the data and indices arrays.
    - shape: Tuple specifying the shape of the matrix (rows, columns).

    Returns:
    - SymPy SparseMatrix object.
    """
    # Construct the SymPy SparseMatrix
    return SparseMatrix(shape[0], shape[1], { (i, indices[j]): data[j] for i in range(shape[0]) for j in range(indptr[i], indptr[i+1]) })

# Example CSR array
data = [1, 2, 3, 4, 5, 6]
indices = [0, 2, 1, 2, 0, 1]
indptr = [0, 2, 3, 6]
shape = (3, 3)

# Convert CSR array to SymPy SparseMatrix
sparse_matrix = csr_to_sparse_matrix(data, indices, indptr, shape)

# Print the SymPy SparseMatrix
print("SymPy SparseMatrix:")
print(sparse_matrix)


SymPy SparseMatrix:
Matrix([[1, 0, 2], [0, 3, 0], [5, 6, 4]])


In [41]:
import numpy as np
from scipy.sparse import csc_matrix
from sympy import SparseMatrix


# Convert SciPy sparse matrix to SymPy SparseMatrix
D1_C2_sympy = SparseMatrix(D1_C2.toarray())

# Print the SymPy SparseMatrix
print(D1_C2_sympy)


SymPy SparseMatrix:
Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, -0.500000000000000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [44]:
def H_spin(N, J):
    """
    Compute the Hamiltonian of an N-spin system with nearest-neighbor interactions.

    Parameters
    ----------
    N : int
        The number of spins in the system.
    J : float
        The coupling constant.

    Returns
    -------
    scipy.sparse.csr_matrix
        The Hamiltonian of the system.
    """
    H = 0

    for i in range(1, N+1):
        if i == N:
            ip1 = 1
        else:
            ip1 = i + 1

        sx_i = scp.sparse.load_npz(f'results/time_dependent_GGE/heisenberg_spins/operators/N_{N}/sx_k_{i}.npz') 
        sx_ip1 = scp.sparse.load_npz(f'results/time_dependent_GGE/heisenberg_spins/operators/N_{N}/sx_k_{ip1}.npz')

        sy_i = scp.sparse.load_npz(f'results/time_dependent_GGE/heisenberg_spins/operators/N_{N}/sy_k_{i}.npz')
        sy_ip1 = scp.sparse.load_npz(f'results/time_dependent_GGE/heisenberg_spins/operators/N_{N}/sy_k_{ip1}.npz')

        sz_i = scp.sparse.load_npz(f'results/time_dependent_GGE/heisenberg_spins/operators/N_{N}/sz_k_{i}.npz')
        sz_ip1 = scp.sparse.load_npz(f'results/time_dependent_GGE/heisenberg_spins/operators/N_{N}/sz_k_{ip1}.npz')

        H += J * (sx_i @ sx_ip1 + sy_i @ sy_ip1 + sz_i @ sz_ip1)

    return H

N = 8
J = 1

C2 = H_spin(N, J)
C2.toarray()

array([[2. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 1. +0.j, 0.5+0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j, 1. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       ...,
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 1. +0.j, 0.5+0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0.5+0.j, 1. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 2. +0.j]])

In [46]:
(C2 @ C2).toarray()

array([[4. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 1.5+0.j, 1. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 1. +0.j, 1.5+0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       ...,
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 1.5+0.j, 1. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 1. +0.j, 1.5+0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 4. +0.j]])

In [51]:
x = (C2 @ C2 @ C2 @ C2).toarray()
np.count_nonzero(x - np.diag(np.diagonal(x)))

9726